In [ ]:
import tkinter as tk
from tkinter import ttk
import weaviate
from dotenv import load_dotenv
import os

load_dotenv()

class WeaviateGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Movie recommendation GUI")
        self.root.configure(bg="#8f5179") 

        self.style = ttk.Style()
                
        ## -- query text entry -- ##
        self.query_entry = ttk.Entry(self.root)
        self.query_entry.grid(row=0, column=1, padx=10, pady=10)
        
        ## -- combobox for choosing search type -- ##
        self.search_type_combobox = ttk.Combobox(self.root, values=["Title", "Tag"], state="readonly",width=10)
        self.search_type_combobox.current(0)  # Imposta il valore predefinito
        self.search_type_combobox.grid(row=0, column=0, padx=5, pady=5)
        
        ## -- search button that calls search_movies -- ##
        self.search_button = ttk.Button(self.root, text="Search", command=self.search_movies)
        self.search_button.grid(row=0, column=2, padx=5, pady=5)
        
        ## -- output field -- ##
        self.results_text = tk.Text(self.root, height=50, width=120, state=tk.DISABLED)
        self.results_text.grid(row=1, column=0, columnspan=3, padx=5, pady=5)
        
        self.scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=self.results_text.yview)
        self.scrollbar.grid(row=1, column=3, sticky="ns")
        self.results_text.configure(yscrollcommand=self.scrollbar.set)

        self.client = weaviate.connect_to_wcs(
            cluster_url=os.getenv("WCD_URL"),
            auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WCD_API_KEY")),
            headers={
               "X-Cohere-Api-Key": os.environ["COHERE_APIKEY"] 
            }
        )
        self.client.close()
    
    ## -- lookup the search type and call the right function -- #
    def search_movies(self):
        query = self.query_entry.get()
        search_type = self.search_type_combobox.get()
        if search_type == "Title":
            results = self.search_movies_query_by_title(query)
        elif search_type == "Tag":
            results = self.search_movies_query_by_tag(query)
    
    ## -- search by title: retrieve tags, generative near text search for those tags -- ##
    def search_movies_query_by_title(self, query):
        try:
            self.client.connect()
            movies_collection = self.client.collections.get("Movies_new")
            response = movies_collection.query.near_text(
                query=query,
                target_vector="title",
                limit=1,
            )
            response_tag = movies_collection.generate.near_text(
                query= str(response.objects[0].properties['tag']),
                target_vector="tag",
                limit=3,
                single_prompt= "Write a short paragraph on this movie : {title}"
            )
            
            self.display_movies(response_tag.objects[1:]) # exclude the first one
        finally:
            self.client.close()
    
    ## -- search by tags: generative near text search for tags -- ##
    def search_movies_query_by_tag(self, query):
        try:
            self.client.connect()
            movies_collection = self.client.collections.get("Movies_new")
            response_tag = movies_collection.generate.near_text(
                query= query,
                target_vector="tag",
                limit=2,
                single_prompt= "Write a short paragraph on this movie : {title}"
            )
            self.display_movies(response_tag.objects)
        finally:
            self.client.close()
    
    ## -- display recommendations -- #
    def display_movies(self,response):
        text = ""
        for i, obj in enumerate(response):
            text += f"{obj.properties['title']} , {obj.properties['genres']}\n{obj.generated}\n\n"
            
        self.results_text.config(state=tk.NORMAL)
        self.results_text.delete('1.0', tk.END)
        self.results_text.insert(tk.END, text)
        self.results_text.config(state=tk.DISABLED)


if __name__ == "__main__":
    root = tk.Tk()
    app = WeaviateGUI(root)
    root.mainloop()
